# Импорт библиотек и загрузка данных

In [1]:
import os
import pickle
import scipy
import pandas as pd
import numpy as np
import implicit

from tqdm import tqdm_notebook

In [2]:
df = pd.read_csv('C:/Users/potopnins/Проект1.csv', sep=';', engine= 'python', parse_dates=['Дата_Создания'])

# Data Preprocessing

In [3]:
#код подробно рассмотрен в ноутбуке 0_EDA
df['Оборот'] = df['Оборот'].str.replace('[A-Za-z]', '').str.replace(',', '.').astype(float)
df_shape_first = df.shape[0]
df = df[(df.Оборот>1)&(df.Количество>0)]
df['Год'] = pd.DatetimeIndex(df['Дата_Создания']).year
df['Месяц'] = pd.DatetimeIndex(df['Дата_Создания']).month
df['День_недели']=df['Дата_Создания'].dt.weekday
df['День_недели'] = df['День_недели'].replace({0:'Пон', 1:'Вт',2:'Ср', 3:'Чт', 4:'Пт', 5:'Сб', 6:'Вс'})
#df = df.reset_index().sort_values(by=['Клиент_Но', 'Документ_Но', 'Дата_Создания', 'Товар_Но', 'РО_Отдел', 'index']).set_index('index')

In [4]:
#проверяем потери
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2774411 entries, 0 to 2904804
Data columns (total 15 columns):
 #   Column               Dtype         
---  ------               -----         
 0   Клиент_Но            object        
 1   Документ_Но          object        
 2   Дата_Создания        datetime64[ns]
 3   Товар_Но             object        
 4   Наименование_Товара  object        
 5   Бренд                object        
 6   Merdis_ПодГруппа     object        
 7   Merdis_Группа        object        
 8   Рег_Офис             object        
 9   РО_Отдел             object        
 10  Оборот               float64       
 11  Количество           int64         
 12  Год                  int64         
 13  Месяц                int64         
 14  День_недели          object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(10)
memory usage: 338.7+ MB


# Подготовка данных для модели

from sklearn import preprocessing
#Выбираем столбцы с типом object
objFeatures = df.select_dtypes(include="object").columns

#Делаем цикл чтобы закодировать данные с типом object
le = preprocessing.LabelEncoder()

for feat in objFeatures:
    df[feat] = le.fit_transform(df[feat].astype(str))

In [5]:
#будем рекомендовать современное
contemporary_items = df[df["Дата_Создания"] > 
                                  "2019-05-15 00:00:00"]["Товар_Но"].unique()
df = df[df["Товар_Но"].isin(contemporary_items)]

In [6]:
# выберем только пользователей с более, чем одной транзакцией
transactions_cnt = df\
                    .groupby(by=["Клиент_Но"])["Документ_Но"]\
                    .count()\
                    .reset_index()

multi_trans_users = transactions_cnt[transactions_cnt["Документ_Но"] > 1]["Клиент_Но"]

In [7]:
#Выделяем рандомно тестовых юзеров, клиенты которые купили больше одного раза
test_users = np.random.choice(multi_trans_users, 10000)

In [8]:
#собираем train test на основе наших юзеров
train, test = df[~df["Клиент_Но"].isin(test_users)], \
              df[df["Клиент_Но"].isin(test_users)]

In [9]:
#записываем последние транзакции
last_transactions = test.drop_duplicates(subset="Клиент_Но", keep="last")["Документ_Но"]

In [10]:
#данные для теста и валидации
test_data = test[~test["Документ_Но"].isin(last_transactions)] #
test_validation = test[test["Документ_Но"].isin(last_transactions)] #данные на валидации

In [11]:
# клиенты только из train, а продукты из всего набора данных
def enumerated_dict(values):
    enum_dict = {}
    reverse_dict = {}
    
    for n, value in enumerate(values):
        enum_dict[value] = n
        reverse_dict[n] = value
        
    return enum_dict, reverse_dict


def predict_user(model, user_id, products, product_dict, reverse_product_dict, matrix_shape):
    enum_clients = np.zeros(len(products))
    enum_products = np.array([product_dict[product] for product in products])

    sparse_matrix = scipy.sparse.csr_matrix((np.ones(shape=(len(enum_clients))), 
                                             (enum_clients, enum_products)), 
                                            shape=matrix_shape)
    
    rec = model.recommend(0, sparse_matrix, N=30, recalculate_user=True,
                     filter_already_liked_items=False)
    
    return [[user_id, reverse_product_dict[r[0]]] for r in rec]

client_dict, reverse_client_dict = enumerated_dict(df["Клиент_Но"].unique())
product_dict, reverse_product_dict = enumerated_dict(df["Товар_Но"].unique())

In [12]:
# Определим размер матрицы
matrix_shape = (max(reverse_client_dict.keys()) + 1, max(reverse_product_dict.keys()) + 1)

In [13]:
enum_clients = np.array([client_dict[client] for client in train["Клиент_Но"]])
enum_products = np.array([product_dict[product] for product in train["Товар_Но"]])

sparse_matrix = scipy.sparse.coo_matrix((np.ones(shape=(len(enum_clients))), 
                                         (enum_clients, enum_products)), 
                                        shape=matrix_shape)
print("Sparticity: ", 100 - df.shape[0] / \
        (sparse_matrix.shape[0] * sparse_matrix.shape[1]))

Sparticity:  99.99963976242498


# Implicite
- Данная библиотека активно развиваеться и подходит для данных с неявной обратной связью

# nearest_neighbours

In [14]:
# Initialize model
model = implicit.nearest_neighbours.CosineRecommender(K=1)

# Fit model
model.fit((sparse_matrix.T))

In [15]:
# Рекомендации для отсутствующих пользователей
recommendations = []

for test_client in tqdm_notebook(test_data["Клиент_Но"].unique()):
    products = test_data[test_data["Клиент_Но"]==test_client]["Товар_Но"]
    rec = predict_user(model, test_client, products, product_dict, reverse_product_dict,
                       (1, matrix_shape[1]))
    recommendations.extend(rec)

C:\Users\potopnins\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [16]:
# датафрейм с покупками в реальности
reality = test_validation[["Клиент_Но", "Товар_Но"]].copy()
reality.loc[:, "is_buyed"] = 1

# Metrics (precision@30, avg_precision@30, average_normed_precision@30)

In [17]:
rec_df = pd.DataFrame(recommendations, columns=["Клиент_Но", "Товар_Но"])\
            .merge(reality, 
                   on=["Клиент_Но", "Товар_Но"], 
                   how="left", 
                   sort=False)\
            .fillna(0)

In [18]:
# словарь с количеством покупок на валидации
real_dict = reality.groupby(by="Клиент_Но")["is_buyed"].sum().to_dict()

In [19]:
# add zeros to k items length
def add_to_k(lst, k):
    return lst + [0] * max(k - len(lst), 0)

# precision at k
def precision_at_k(r_true_arr, k):
    return np.sum(r_true_arr[:k]) / k


# average precision at k
def average_precision_at_k(r_true_arr, k):
    apk = 0
    for n in range(0, k):
        apk += precision_at_k(r_true_arr, n + 1) * r_true_arr[n]
    if np.sum(r_true_arr[:k]) != 0:
        return (apk) / k
    else:
        return 0


# average normed precision at k
def average_normed_precision_at_k(r_true_arr, k, n_true):
    apk = 0
    apk_ideal = n_true / k
    
    for n in range(0, k):
        apk += precision_at_k(r_true_arr, n + 1) * r_true_arr[n]
    if np.sum(r_true_arr[:k]) != 0:
        return ((apk) / k) / apk_ideal
    else:
        return 0

In [20]:
#np.mean([precision_at_k(i, 30) for i in 
         #rec_df.groupby(by="Клиент_Но", sort=False)["is_buyed"].apply(list)])

In [21]:
#np.mean([average_precision_at_k(add_to_k(i, 30), 30) for client, i in 
         #rec_df.groupby(by="Клиент_Но")["is_buyed"].apply(list).reset_index().values])

In [22]:
np.mean([average_normed_precision_at_k(add_to_k(i, 30), 30, real_dict.get(client, 0)) for client, i in 
         rec_df.groupby(by="Клиент_Но")["is_buyed"].apply(list).reset_index().values])

0.0847070671852897

### Рекомендации для пользователей которые ранее покупали

In [77]:
reality.sample(5)

,Клиент_Но,Товар_Но,is_buyed
1776099,IE455175,643649,1
2894846,IX908686,1385600,1
1326604,ID285336,607453,1
777939,I1778669,1179570,1
137204,I0274391,1186000,1


 - https://www.citilink.ru/catalog/computers_and_notebooks/monitors_and_office/cartridges/643649/
 - https://www.citilink.ru/catalog/large_and_small_appliances/small_appliances/kettles/1385600/
 - Эти и остальные позиции можно найти на сайте https://www.citilink.ru/

### Рекомендации для пользователей которые ранее не покупали

In [71]:
recommendations[5:100:20]

[['I0002274', '933473'],
 ['I0003776', '1083044'],
 ['I0003776', '1052926'],
 ['I0004296', '1130165'],
 ['I0011861', '1366631']]

# ALS

In [ ]:
def predict_user(model_ALS, user_id, products, product_dict, reverse_product_dict, matrix_shape):
    enum_clients = np.zeros(len(products))
    enum_products = np.array([product_dict[product] for product in products])

    sparse_matrix = scipy.sparse.csr_matrix((np.ones(shape=(len(enum_clients))), 
                                             (enum_clients, enum_products)), 
                                            shape=matrix_shape)
    
    rec = model_ALS.recommend(0, sparse_matrix, N=30, recalculate_user=True,
                     filter_already_liked_items=False)
    
    return [[user_id, reverse_product_dict[r[0]]] for r in rec]

client_dict, reverse_client_dict = enumerated_dict(df["Клиент_Но"].unique())
product_dict, reverse_product_dict = enumerated_dict(df["Товар_Но"].unique())

In [ ]:
# Initialize model
model_ALS = implicit.als.AlternatingLeastSquares(factors=30)

# Fit model
model_ALS.fit((sparse_matrix.T))

In [ ]:
# Рекомендации для отсутствующих пользователей
recommendations = []

for test_client in tqdm_notebook(test_data["Клиент_Но"].unique()):
    products = test_data[test_data["Клиент_Но"]==test_client]["Товар_Но"]
    rec = predict_user(model_ALS, test_client, products, product_dict, reverse_product_dict,
                       (1, matrix_shape[1]))
    recommendations.extend(rec)

In [ ]:
# датафрейм с покупками в реальности
reality = test_validation[["Клиент_Но", "Товар_Но"]].copy()
reality.loc[:, "is_buyed"] = 1

In [ ]:
rec_df = pd.DataFrame(recommendations, columns=["Клиент_Но", "Товар_Но"])\
            .merge(reality, 
                   on=["Клиент_Но", "Товар_Но"], 
                   how="left", 
                   sort=False)\
            .fillna(0)

In [ ]:
# словарь с количеством покупок на валидации
real_dict = reality.groupby(by="Клиент_Но")["is_buyed"].sum().to_dict()

In [ ]:
#Метрика
np.mean([average_normed_precision_at_k(add_to_k(i, 30), 30, real_dict.get(client, 0)) for client, i in 
         rec_df.groupby(by="Клиент_Но")["is_buyed"].apply(list).reset_index().values])

# Сохраняем лучшу модель

In [ ]:
enum_clients = np.array([client_dict.get(client, 0) for client in df["Клиент_Но"]])
enum_products = np.array([product_dict.get(product, 0) for product in df["Товар_Но"]])

sparse_matrix = scipy.sparse.coo_matrix((np.ones(shape=(len(enum_clients))), 
                                         (enum_clients, enum_products)), 
                                        shape=matrix_shape)
print("Sparticity: ", 100 - df.shape[0] / \
        (sparse_matrix.shape[0] * sparse_matrix.shape[1]))

In [ ]:
# Initialize model
model = implicit.nearest_neighbours.CosineRecommender(K=1)

# Fit model
model.fit((sparse_matrix.T))

In [ ]:
# сохраняем помимо модели еще и словари, чтобы была возможность создать матрицу
with open("citilink_implicit.pkl", "wb") as f:
    pickle.dump((model, client_dict, reverse_client_dict, 
                 product_dict, reverse_product_dict), f)

# Summary:
**1. Построили две модели на основе библиотеки implicit**

**2. Сделали валидационное и обучающее множества, получив на валидации результаты:**
    - ALS NMAP@K: 0.57 на соревновании min 0.63**
    - Ближайшие соседи NMAP@K: 0.83 на соревновании min 0.63**
**3. Проверили рекомендации на сайте :)**